# Workflow Integration

> End-to-end workflow: Finnish texts → Translation → Audio → Anki cards

In [ ]:
#| default_exp workflow

In [ ]:
#| export
from pathlib import Path

from suomi.core import *
from suomi.tsv import *
from suomi.mp3 import *
from suomi.anki import *

## Main Workflow Function

In [ ]:
#| export
def create_cards(
    texts: list[str],
    deck: str,
    tags: str | list[str] = "lang::fi",
    output_dir: str = "output",
    overwrite: bool = False,
) -> None:
    """
    Create Anki cards from Finnish texts (end-to-end workflow).
    
    Steps:
    1. Translate Finnish → English/Japanese (OpenAI API)
    2. Generate audio files (Piper TTS)
    3. Update TSV with file paths
    4. Upload to Anki (AnkiConnect)
    
    Args:
        texts: List of Finnish words/phrases to create cards for
        deck: Anki deck name (e.g., "06::Daily")
        tags: Tags for the cards (string or list), hierarchical format supported
              Examples: "src::daily", "src::class,level::A1", ["src::medical", "urgent"]
              Note: "lang::fi" is always auto-included
        output_dir: Directory for TSV and audio files (default: "output")
        overwrite: If True, delete existing deck and overwrite TSV file
    
    Example:
        >>> create_cards(
        ...     texts=["kissa", "koira"],
        ...     deck="06::Daily",
        ...     tags="src::daily"
        ... )
    """
    
    tsv_path = f"{output_dir}/{Path(deck.replace("::", "_")).stem + ".tsv"}"
    if overwrite:
        if deck in deckNames():
            deleteDeck(deck)
        # TSV will be auto-overwritten by texts2tsv()
    else:
        if Path(tsv_path).exists():
            raise FileExistsError(
                f"TSV file already exists: {tsv_path}\n"
                f"Use overwrite=True to replace it."
            )
        if deck in deckNames():
            raise ValueError(
                f"Deck '{deck}' already exists in Anki.\n"
                f"Use overwrite=True to replace it."
            )
        
    audio_dir = f"{output_dir}/audio"
    images_dir = f"{output_dir}/images"
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    texts2tsv(texts, tsv_path, tags=tags)
    mp3s(tsv_path, output_dir=audio_dir)
    update_tsv_media_paths(tsv_path, dirs=[audio_dir, images_dir])
    addnotes(deck, tsv_path)

## Example Usage

In [ ]:
txt = "Ahaa!::05::Kasvot.txt"
tsv = Path(txt).with_suffix(".tsv")
tsv, tsv.stem

(PosixPath('Ahaa!::05::Kasvot.tsv'), 'Ahaa!::05::Kasvot')

In [ ]:
#| eval: false
fn = "Ahaa!::05::Lääkarissä.txt"
create_cards(
    texts=cattxt(fn),
    deck=Path(fn).stem,
    tags="lang::fi::Lääkarissä,Ahaa!::05::Lääkarissä",
    overwrite=True,
)

tsv = f"output/{Path(fn.replace("::", "_")).with_suffix('.tsv')}"
cattsv(tsv)[0][:2]

[{'Finnish': 'Nasim istuu ja odottaa lääkäriä, ja hän on väsynyt, koska hän nukkuu huonosti joka yö.',
  'English': 'Nasim is sitting and waiting for the doctor, and he is tired because he sleeps poorly every night.',
  'Japanese': 'ナシムは座って医者を待っていて、毎晩よく眠れないので疲れています。',
  'mp3_path': 'output/audio/Ahaa!_05_Lääkarissä_00.mp3',
  'img_path': '',
  'tags': 'lang/fi,lang::fi::Lääkarissä,Ahaa!::05::Lääkarissä'},
 {'Finnish': 'On vaikea opiskella suomea, kun on väsynyt, ja vaikka kurssilla Nasim ymmärtää sanat, hän unohtaa ne kotona.',
  'English': "It's hard to study Finnish when you're tired, and even though Nasim understands the words in class, he forgets them at home.",
  'Japanese': '疲れているとフィンランド語を勉強するのは難しく、授業ではナシムは単語を理解していても、家では忘れてしまいます。',
  'mp3_path': 'output/audio/Ahaa!_05_Lääkarissä_01.mp3',
  'img_path': '',
  'tags': 'lang/fi,lang::fi::Lääkarissä,Ahaa!::05::Lääkarissä'}]

In [ ]:
#| eval: false
fn = "Ahaa!::05::Kasvot.txt"
create_cards(
    texts=cattxt(fn),
    deck=Path(fn).stem,
    tags="lang::fi::Kasvot,Ahaa!::05::Lääkarissä",
    overwrite=True,
)

tsv = f"output/{Path(fn.replace("::", "_")).with_suffix('.tsv')}"
cattsv(tsv)[0][:2]

[{'Finnish': 'Missä sattuu? — Mulla on kasvot kipeät.',
  'English': 'Where does it hurt? — My face hurts.',
  'Japanese': 'どこが痛いですか？ — 顔が痛いです。',
  'mp3_path': 'output/audio/Ahaa!_05_Kasvot_00.mp3',
  'img_path': 'output/images/Ahaa!_05_Kasvot.jpg',
  'tags': 'lang/fi,lang::fi::Kasvot,Ahaa!::05::Lääkarissä'},
 {'Finnish': 'Mikä vaivaa? — Mulla lähtee hiukset.',
  'English': "What's wrong? — I'm losing my hair.",
  'Japanese': 'どうしたの？ — 髪が抜けています。',
  'mp3_path': 'output/audio/Ahaa!_05_Kasvot_01.mp3',
  'img_path': 'output/images/Ahaa!_05_Kasvot.jpg',
  'tags': 'lang/fi,lang::fi::Kasvot,Ahaa!::05::Lääkarissä'}]

In [ ]:
#| eval: false
fn = "Ahaa!::05::Oireet.txt"
create_cards(
    texts=cattxt(fn),
    deck=Path(fn).stem,
    tags="lang::fi::Oireet,Ahaa!::05::Lääkarissä",
    overwrite=True,
)

tsv = f"output/{Path(fn.replace("::", "_")).with_suffix('.tsv')}"
cattsv(tsv)[0][:2]

[{'Finnish': 'Mitä kuuluu? — Ihan hyvää, kiitos.',
  'English': "How are you? — I'm doing quite well, thank you.",
  'Japanese': 'お元気ですか？ — とても元気です、ありがとう。',
  'mp3_path': 'output/audio/Ahaa!_05_Oireet_00.mp3',
  'img_path': '',
  'tags': 'lang/fi,lang::fi::Oireet,Ahaa!::05::Lääkarissä'},
 {'Finnish': 'Mitä kuuluu? — Ei kovin hyvää.',
  'English': 'How are you? — Not very well.',
  'Japanese': 'お元気ですか？ — あまり良くありません。',
  'mp3_path': 'output/audio/Ahaa!_05_Oireet_01.mp3',
  'img_path': '',
  'tags': 'lang/fi,lang::fi::Oireet,Ahaa!::05::Lääkarissä'}]

In [ ]:
#| eval: false
fn = "Vartalo::Jooga.txt"
create_cards(
    texts=cattxt(fn),
    deck=Path(fn).stem,
    tags="lang::fi::Vartalo,Jooga",
    overwrite=True,
)

tsv = f"output/{Path(fn.replace("::", "_")).with_suffix('.tsv')}"
cattsv(tsv)[0][:2]

[{'Finnish': 'Hengitä sisään ja pidennä koko vartaloa, hengitä ulos ja rentouta vartalo, älä jännitä koko kehoa.',
  'English': 'Breathe in and lengthen your whole body, breathe out and relax your body, do not tense your entire body.',
  'Japanese': '息を吸って全身を伸ばし、息を吐いて体をリラックスさせて、全身を緊張させないでください。',
  'mp3_path': 'output/audio/Vartalo_Jooga_00.mp3',
  'img_path': 'output/images/Vartalo_Jooga.jpg',
  'tags': 'lang/fi,lang::fi::Vartalo,Jooga'},
 {'Finnish': 'Hengitä ulos ja pidä pää rentona, pidennä niskaa rauhassa, älä työnnä päätä eteen.',
  'English': 'Breathe out and keep your head relaxed, gently lengthen your neck, do not push your head forward.',
  'Japanese': '息を吐いて頭をリラックスさせ、ゆっくりと首を伸ばし、頭を前に押し出さないでください。',
  'mp3_path': 'output/audio/Vartalo_Jooga_01.mp3',
  'img_path': 'output/images/Vartalo_Jooga.jpg',
  'tags': 'lang/fi,lang::fi::Vartalo,Jooga'}]

In [ ]:
#| eval: false
fn = "Kuukaudet_Vuodenajat.txt"
create_cards(
    texts=cattxt(fn),
    deck=Path(fn).stem,
    tags="lang::fi::Kuukaudet,lang::fi::Vuodenajat",
    overwrite=True,
)

tsv = f"output/{Path(fn.replace("::", "_")).with_suffix('.tsv')}"
cattsv(tsv)[0][:2]

[{'Finnish': 'Tammikuu on talvi ja on yleensä hyvin kylmä.',
  'English': 'January is winter and is usually very cold.',
  'Japanese': '1月は冬で、通常とても寒いです。(いちがつはふゆで、つうじょうとてもさむいです。)',
  'mp3_path': 'output/audio/Kuukaudet_Vuodenajat_00.mp3',
  'img_path': 'output/images/Kuukaudet_Vuodenajat.jpg',
  'tags': 'lang/fi,lang::fi::Kuukaudet,lang::fi::Vuodenajat'},
 {'Finnish': 'Helmikuu on talvea, ja päivät alkavat vähän pidetä.',
  'English': 'February is still winter, and the days start to get a bit longer.',
  'Japanese': '2月はまだ冬で、日が少しず長くなり始めます。(にがつはまだふゆで、ひがすこしながくなりはじめます。)',
  'mp3_path': 'output/audio/Kuukaudet_Vuodenajat_01.mp3',
  'img_path': 'output/images/Kuukaudet_Vuodenajat.jpg',
  'tags': 'lang/fi,lang::fi::Kuukaudet,lang::fi::Vuodenajat'}]

In [ ]:
#| eval: false
fn = "Järjestysluvut.txt"
create_cards(
    texts=cattxt(fn),
    deck=Path(fn).stem,
    tags="lang::fi::Järjestysluvut",
    overwrite=True,
)

tsv = f"output/{Path(fn.replace("::", "_")).with_suffix('.tsv')}"
cattsv(tsv)[0][:2]

[{'Finnish': 'Mikä kerros? - Missä kerroksessa? ensimmäinen kerros – ensimmäisessä kerroksessa. Mikä on ensimmäinen kuukausi? Ensimmäinen kuukausi on tammikuu.',
  'English': 'Which floor? - On which floor? first floor – on the first floor. What is the first month? The first month is January.',
  'Japanese': '何階ですか？ - どの階ですか？ 一階 - 一階にいます。最初の月は何ですか？最初の月は1月です。',
  'mp3_path': 'output/audio/Järjestysluvut_00.mp3',
  'img_path': 'output/images/Järjestysluvut.jpg',
  'tags': 'lang/fi,lang::fi::Järjestysluvut'},
 {'Finnish': 'Mikä kerros? - Missä kerroksessa? toinen kerros – toisessa kerroksessa. Mikä on toinen kuukausi? Toinen kuukausi on helmikuu.',
  'English': 'Which floor? - On which floor? second floor – on the second floor. What is the second month? The second month is February.',
  'Japanese': '何階ですか？ - どの階ですか？ 二階 - 二階にいます。二番目の月は何ですか？二番目の月は2月です。',
  'mp3_path': 'output/audio/Järjestysluvut_01.mp3',
  'img_path': 'output/images/Järjestysluvut.jpg',
  'tags': 'lang/fi,lang::fi::Järjest

In [ ]:
#| eval: false
fn = "Yksikkö_Monikko.txt"
create_cards(
    texts=cattxt(fn),
    deck=Path(fn).stem,
    tags="lang::fi::Monikko",
    overwrite=True,
)

tsv = f"output/{Path(fn.replace("::", "_")).with_suffix('.tsv')}"
cattsv(tsv)[0][:2]

[{'Finnish': 'Opiskelija on koulussa ja opiskelijat ovat koulussa.',
  'English': 'The student is at school and the students are at school.',
  'Japanese': '学生は学校にいます、そして学生たちは学校にいます。',
  'mp3_path': 'output/audio/Yksikkö_Monikko_00.mp3',
  'img_path': '',
  'tags': 'lang/fi,lang::fi::Monikko'},
 {'Finnish': 'Opiskelija on suomalainen ja opiskelijat ovat suomalaiset.',
  'English': 'The student is Finnish and the students are Finnish.',
  'Japanese': '学生はフィンランド人です、そして学生たちはフィンランド人です。',
  'mp3_path': 'output/audio/Yksikkö_Monikko_01.mp3',
  'img_path': '',
  'tags': 'lang/fi,lang::fi::Monikko'}]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()